In [163]:
import pandas as pd 
import geopandas as gpd 
import numpy as np 
import json 
from glob import glob 

import sys 
sys.path.append("../")
from logger import setup_logger
logger = setup_logger("analysis-df-assembly")
logger.setLevel("INFO")

WGS='EPSG:4326'
PROJ='EPSG:2263'

import os 

logger.info("Modules loaded.")



2024-10-23 11:49:30 - analysis-df-assembly - INFO - Modules loaded.


In [164]:
INTERNAL_DATA = True 

USE_SMOOTHING = True


In [165]:
ICAR_NONE_RUN='../runs/icar_none/simulated_False/ahl_True/20241021-1038'
ICAR_CHEATING_RUN='../runs/icar_cheating/simulated_False/ahl_True/20241022-1130'

In [166]:
ICAR_NONE_ESTIMATES = glob(f"{ICAR_NONE_RUN}/estimate*.csv")
ICAR_CHEATING_ESTIMATES = glob(f"{ICAR_CHEATING_RUN}/estimate*.csv")
logger.info(f"Found {len(ICAR_NONE_ESTIMATES)} ICAR_NONE estimates and {len(ICAR_CHEATING_ESTIMATES)} ICAR_CHEATING estimates.")

2024-10-23 11:49:30 - analysis-df-assembly - INFO - Found 2 ICAR_NONE estimates and 3 ICAR_CHEATING estimates.


In [167]:
icar_cheating_estimates = {} 
for f in ICAR_CHEATING_ESTIMATES:
    df = pd.read_csv(f)
    df['tract_id'] = df['tract_id'].astype(int).astype(str)
    icar_cheating_estimates[os.path.splitext(os.path.basename(f))[0]] = df


In [168]:
icar_none_estimates = {} 
for f in ICAR_NONE_ESTIMATES:
    df = pd.read_csv(f)
    df['tract_id'] = df['tract_id'].astype(int).astype(str)
    icar_none_estimates[os.path.splitext(os.path.basename(f)[0])] = df
    

In [169]:

if USE_SMOOTHING: 
    icar_model_estimates = icar_cheating_estimates
    logger.info("Using smoothed estimates.")
else:
    icar_model_estimates = icar_none_estimates
    logger.info("Using unsmoothed estimates.")

2024-10-23 11:49:31 - analysis-df-assembly - INFO - Using smoothed estimates.


In [170]:
ct_nyc = gpd.read_file('geo/data/ct-nyc-wi-2020.geojson')


TO_DROP = ['OBJECTID','BoroCode','CT2020','CDEligibil','NTA2020','CDTA2020','Shape__Area','Shape__Length','geometry']
ct_nyc.drop(columns=TO_DROP, inplace=True)

logger.info(f"Loaded NYC CT shapefile with {len(ct_nyc.index)} CTs.")

2024-10-23 11:49:31 - analysis-df-assembly - INFO - Loaded NYC CT shapefile with 2325 CTs.


In [171]:
ct_nyc.columns

Index(['CTLabel', 'BoroName', 'BoroCT2020', 'NTAName', 'CDTANAME', 'GEOID',
       'PUMA'],
      dtype='object')

In [172]:
ct_nyc_clip = gpd.read_file('geo/data/ct-nyc-2020.geojson')
logger.info(f"Loaded NYC CT (water clipped) shapefile with {len(ct_nyc_clip.index)} CTs.")

2024-10-23 11:49:31 - analysis-df-assembly - INFO - Loaded NYC CT (water clipped) shapefile with 2327 CTs.


In [173]:
ct_nyc = ct_nyc.merge(icar_model_estimates['estimate_p_y'], left_on='GEOID', right_on='tract_id', suffixes=('_ct', '_p_y'))
ct_nyc = ct_nyc.merge(icar_model_estimates['estimate_at_least_one_positive_image_by_area'], left_on='GEOID', right_on='tract_id', suffixes=('_ct', '_p_alop'))
ct_nyc = ct_nyc.merge(icar_model_estimates['estimate_at_least_one_positive_image_by_area_if_you_have_100_images'], left_on='GEOID', right_on='tract_id', suffixes=('_ct', '_p_alop_100'))

# drop empirical_estimate_* cols 
TO_DROP = [c for c in ct_nyc.columns if 'empirical_estimate_' in c]
ct_nyc.drop(columns=TO_DROP, inplace=True)

logger.info(f"Merged NYC CT shapefile with icar model estimates.")

2024-10-23 11:49:31 - analysis-df-assembly - INFO - Merged NYC CT shapefile with icar model estimates.


In [174]:
# Load data
dp05_nyc_md = pd.read_json('demo/data/acs22_dp05_md.json')

# Normalize the 'variables' column in the JSON
dp05_nyc_md = pd.json_normalize(dp05_nyc_md['variables']).set_index(dp05_nyc_md.index)

# Parse out the 'label' column
# In all rows of the 'label', get the lowest and highest number of '!!'
min_sep = min(dp05_nyc_md['label'].apply(lambda x: x.count('!!')))
max_sep = max(dp05_nyc_md['label'].apply(lambda x: x.count('!!')))

# Create 'desc_i' columns for each level of '!!'
for i in range(min_sep + 1, max_sep + 2):  # Adjusting range to account for correct indexing
    dp05_nyc_md[f'desc_{i}'] = dp05_nyc_md['label'].apply(
        lambda x: x.split('!!')[i-1] if len(x.split('!!')) >= i else None
    )

# drop TO_DROP 
TO_DROP = ['label','concept','predicateType','group','limit','predicateOnly']
dp05_nyc_md = dp05_nyc_md.drop(columns=TO_DROP)

desc_1_filter = ['Estimate']
dp05_nyc_md = dp05_nyc_md[dp05_nyc_md['desc_1'].isin(desc_1_filter)]

# Output the modified dataframe
# display all rows 
dp05_nyc_md = dp05_nyc_md.sort_index()

In [175]:
dp05_nyc = pd.read_json('demo/data/acs22_dp05.json', orient='records')

dp05_nyc.columns = dp05_nyc.iloc[0]
dp05_nyc = dp05_nyc[1:]

dp05_nyc['tract_id'] = dp05_nyc['GEO_ID'].str.split('US', expand=True)[1]

RACE_COLS = {
    'DP05_0079E': 'nhl_white_alone', 
    'DP05_0080E': 'nhl_black_alone', 
    'DP05_0073E': 'hispanic_alone', 
    'DP05_0082E': 'nhl_asian_alone'
}

race_nyc = dp05_nyc[list(RACE_COLS.keys())]
race_nyc.columns = race_nyc.columns.map(lambda x: RACE_COLS[x])
race_nyc.index = dp05_nyc['tract_id']
race_nyc 

,nhl_white_alone,nhl_black_alone,hispanic_alone,nhl_asian_alone
tract_id,,,,
36005000100,1098,2000,1172,123
36005000200,83,1281,3109,299
36005000400,283,1559,4212,103
36005001600,106,2132,3507,148
36005001901,306,942,842,0
...,...,...,...,...
36085030302,2209,1568,1625,918
36085031901,289,1626,1469,224
36085031902,473,2388,1913,217


In [176]:
ct_nyc = ct_nyc.merge(race_nyc, left_on='GEOID', right_index=True)  

In [177]:
ct_nyc = ct_nyc.set_index('GEOID')

In [178]:
gdf_ct_nyc = gpd.read_file('geo/data/ct-nyc-wi-2020.geojson').to_crs(PROJ)[['GEOID','geometry']]


In [179]:
# FLOODNET 
floodnet_sensor = pd.read_csv('flooding/static/floodnet-flood-sensor-sep-2023.csv', engine='pyarrow')
floodnet_tide = pd.read_csv('flooding/static/floodnet-tide-sep-2023.csv', engine='pyarrow')
floodnet_weather = pd.read_csv('flooding/static/floodnet-weather-sep-2023.csv', engine='pyarrow')


all_floodnet_sensors_geo = pd.concat([floodnet_sensor.groupby('deployment_id').first()[['lat','lon']].reset_index(), floodnet_tide.groupby('sensor_id').first()[['lat','lon']].reset_index(), floodnet_weather.groupby('sensor_id').first()[['lat','lon']].reset_index()], axis=0)

all_floodnet_sensor_geo = gpd.GeoDataFrame(all_floodnet_sensors_geo, geometry=gpd.points_from_xy(all_floodnet_sensors_geo.lon, all_floodnet_sensors_geo.lat), crs='EPSG:4326').to_crs(2263)

del floodnet_sensor, floodnet_tide, floodnet_weather

logger.info("Loaded and processed Floodnet sensor data.")

2024-10-23 11:49:33 - analysis-df-assembly - INFO - Loaded and processed Floodnet sensor data.


In [180]:
# get the number of floodnet sensors in each census tract in gdf_ct_nyc, including tracts with 0 sensors
ct_nyc['n_sensors'] = gpd.sjoin(gdf_ct_nyc, all_floodnet_sensor_geo).groupby('GEOID').size().reindex(ct_nyc.index).fillna(0)

logger.info("Merged Floodnet sensor data with NYC CT shapefile.")
logger.info(ct_nyc['n_sensors'].describe().to_string())



2024-10-23 11:49:33 - analysis-df-assembly - INFO - Merged Floodnet sensor data with NYC CT shapefile.
2024-10-23 11:49:33 - analysis-df-assembly - INFO - count    2325.000000
mean        0.036129
std         0.272811
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         6.000000


In [181]:
# DEP STORMWATER 
dep_light = gpd.read_file('flooding/data/NYCFloodStormwaterFloodMaps/NYC Stormwater Flood Map - Extreme Flood (3.66 inches per hr) with 2080 Sea Level Rise/NYC_Stormwater_Flood_Map_Extreme_Flood_3_66_inches_per_hr_with_2080_Sea_Level_Rise.gdb').to_crs(PROJ)
dep_moderate = gpd.read_file('flooding/data/NYCFloodStormwaterFloodMaps/NYC Stormwater Flood Map - Moderate Flood (2.13 inches per hr) with Current Sea Levels/NYC_Stormwater_Flood_Map_Moderate_Flood_2_13_inches_per_hr_with_Current_Sea_Levels.gdb').to_crs(PROJ)

In [182]:
polygons = {} 
for i, row in dep_light.iterrows():
    idx = 0
    for polygon in row['geometry'].geoms:
        polygons[f'{row["Flooding_Category"]}_{idx}'] = polygon
        idx += 1

# dataframe from dict 
dep_light_flattened = gpd.GeoDataFrame(polygons, index=['geometry']).T
# flooding category is the first part of the index


dep_light_flattened.set_geometry('geometry', inplace=True)
dep_light_flattened.crs = dep_light.crs

dep_light_flattened['Flooding_Category'] = dep_light_flattened.index.str.split('_').str[0].astype(int)

In [183]:
polygons = {}
for i, row in dep_moderate.iterrows():
    idx = 0
    for polygon in row['geometry'].geoms:
        polygons[f'{row["Flooding_Category"]}_{idx}'] = polygon
        idx += 1
    
# dataframe from dict
dep_moderate_flattened = gpd.GeoDataFrame(polygons, index=['geometry']).T

dep_moderate_flattened.set_geometry('geometry', inplace=True)
dep_moderate_flattened.crs = dep_moderate.crs

dep_moderate_flattened['Flooding_Category'] = dep_moderate_flattened.index.str.split('_').str[0].astype(int)

In [184]:
# get the total area of light and moderate flooding in each ct in gdf_ct_nyc 
dep_light_flattened_1 = dep_light_flattened[dep_light_flattened['Flooding_Category'] == 1]
dep_light_flattened_2 = dep_light_flattened[dep_light_flattened['Flooding_Category'] == 2]
dep_light_flattened_3 = dep_light_flattened[dep_light_flattened['Flooding_Category'] == 3]
ct_nyc['dep_light_1_area'] = gpd.overlay(gdf_ct_nyc, dep_light_flattened_1, how='intersection').groupby('GEOID')['geometry'].apply(lambda geom: geom.area.sum()).reindex(ct_nyc.index).fillna(0)
ct_nyc['dep_light_2_area'] = gpd.overlay(gdf_ct_nyc, dep_light_flattened_2, how='intersection').groupby('GEOID')['geometry'].apply(lambda geom: geom.area.sum()).reindex(ct_nyc.index).fillna(0)
ct_nyc['dep_light_3_area'] = gpd.overlay(gdf_ct_nyc, dep_light_flattened_3, how='intersection').groupby('GEOID')['geometry'].apply(lambda geom: geom.area.sum()).reindex(ct_nyc.index).fillna(0)

dep_moderate_flattened_1 = dep_moderate_flattened[dep_moderate_flattened['Flooding_Category'] == 1]
dep_moderate_flattened_2 = dep_moderate_flattened[dep_moderate_flattened['Flooding_Category'] == 2]
ct_nyc['dep_moderate_1_area'] = gpd.overlay(gdf_ct_nyc, dep_moderate_flattened_1, how='intersection').groupby('GEOID')['geometry'].apply(lambda geom: geom.area.sum()).reindex(ct_nyc.index).fillna(0)
ct_nyc['dep_moderate_2_area'] = gpd.overlay(gdf_ct_nyc, dep_moderate_flattened_2, how='intersection').groupby('GEOID')['geometry'].apply(lambda geom: geom.area.sum()).reindex(ct_nyc.index).fillna(0)

logger.info("Merged DEP stormwater data with NYC CT shapefile.")

logger.info("\n"+ct_nyc[['dep_light_1_area','dep_light_2_area','dep_light_3_area','dep_moderate_1_area','dep_moderate_2_area']].describe().to_string())

2024-10-23 11:50:31 - analysis-df-assembly - INFO - Merged DEP stormwater data with NYC CT shapefile.
2024-10-23 11:50:31 - analysis-df-assembly - INFO - 
       dep_light_1_area  dep_light_2_area  dep_light_3_area  dep_moderate_1_area  dep_moderate_2_area
count      2.325000e+03      2.325000e+03      2.325000e+03          2325.000000          2325.000000
mean       1.704254e+05      1.305767e+05      1.375244e+05         33568.736899         17255.142646
std        1.926640e+05      2.015365e+05      7.969362e+05         77947.720054         49174.975593
min        0.000000e+00      0.000000e+00      0.000000e+00             0.000000             0.000000
25%        4.756178e+04      1.274651e+04      0.000000e+00             0.000000             0.000000
50%        1.176228e+05      5.511028e+04      0.000000e+00          5211.095209             0.000000
75%        2.227261e+05      1.648135e+05      0.000000e+00         33654.810667          9705.671683
max        1.653186e+06      

In [185]:
# 311 from September 29 Flood 
nyc311_sep29 = pd.read_csv('flooding/data/nyc311_flooding_sep29.csv').dropna(subset=['latitude','longitude'])

nyc311_sep29 = gpd.GeoDataFrame(nyc311_sep29, geometry=gpd.points_from_xy(nyc311_sep29.longitude, nyc311_sep29.latitude), crs=WGS).to_crs(PROJ)

logger.info("Loaded and processed 311 data from September 29, 2023.")

2024-10-23 11:50:31 - analysis-df-assembly - INFO - Loaded and processed 311 data from September 29, 2023.


In [186]:
nyc311_sep29['descriptor'].value_counts()

descriptor
Sewer Backup (Use Comments) (SA)                    1081
Street Flooding (SJ)                                 625
Catch Basin Clogged/Flooding (Use Comments) (SC)     429
Manhole Overflow (Use Comments) (SA1)                 35
Highway Flooding (SH)                                  1
Name: count, dtype: int64

In [187]:
# for each unique val of descriptor, create a column in gdf_ct_nyc with the count of 311 calls of that descriptor type in each tract 
for descriptor in nyc311_sep29['descriptor'].unique():
    # human-readable column name 
    # remove anything inside () 
    desc = descriptor.split('(')[0].strip()
    desc = desc.lower().replace(' ', '_') + '_311c'

    gdf_ct_nyc[desc] = gdf_ct_nyc['geometry'].apply(lambda x: nyc311_sep29[nyc311_sep29['descriptor'] == descriptor].within(x).sum())

logger.info("Merged 311 data with NYC CT shapefile.")
logger.info(gdf_ct_nyc[[c for c in gdf_ct_nyc.columns if '_311c' in c]].sum())


2024-10-23 11:50:35 - analysis-df-assembly - INFO - Merged 311 data with NYC CT shapefile.
2024-10-23 11:50:35 - analysis-df-assembly - INFO - sewer_backup_311c                    1081
street_flooding_311c                  625
catch_basin_clogged/flooding_311c     429
manhole_overflow_311c                  35
highway_flooding_311c                   1
dtype: int64


In [188]:
# merge ct_nyc with gdf_ct_nyc
ct_nyc = ct_nyc.merge(gdf_ct_nyc, on='GEOID')

In [189]:
COLS_ALLOWED_NA_VALS = ['empirical_estimate']
def na_validation(df, cols_allowed_na_vals):
    for c in df.columns:
        if c in cols_allowed_na_vals:
            continue
        if df[c].isna().sum() > 0:
            logger.error(f"Column {c} has {df[c].isna().sum()} NA values.")
    else: 
        logger.success("No N/A values found in columns.")
na_validation(ct_nyc, COLS_ALLOWED_NA_VALS)

2024-10-23 11:50:46 - analysis-df-assembly - SUCCESS - No N/A values found in columns.


In [199]:
# final cleaning 
TO_DROP = ['tract_id', 'n_images_by_area_']
# drop all columns that match regex of entry in list 
current_cols = ct_nyc.columns
for c in TO_DROP:
    ct_nyc = ct_nyc.loc[:, ~ct_nyc.columns.str.contains(c)]

logger.info(f"Dropped columns: {set(current_cols) - set(ct_nyc.columns)}")

2024-10-23 11:59:34 - analysis-df-assembly - INFO - Dropped columns: {'n_images_by_area_p_alop', 'n_images_by_area_ct'}


In [198]:
ct_nyc.describe()

,p_y,p_y_CI_lower,p_y_CI_upper,n_images_by_area_ct,at_least_one_positive_image_by_area,at_least_one_positive_image_by_area_CI_lower,at_least_one_positive_image_by_area_CI_upper,n_images_by_area_p_alop,empirical_estimate,at_least_one_positive_image_by_area_if_you_have_100_images,...,dep_light_1_area,dep_light_2_area,dep_light_3_area,dep_moderate_1_area,dep_moderate_2_area,sewer_backup_311c,street_flooding_311c,catch_basin_clogged/flooding_311c,manhole_overflow_311c,highway_flooding_311c
count,2325.000000,2325.000000,2325.000000,2325.000000,2325.000000,2325.000000,2325.000000,2325.000000,2319.000000,2325.000000,...,2.325000e+03,2.325000e+03,2.325000e+03,2325.000000,2325.000000,2325.000000,2325.000000,2325.000000,2325.000000,2325.000000
mean,0.003020,0.001110,0.006660,398.370753,0.492648,0.299645,0.713894,398.370753,0.001591,0.290829,...,1.704254e+05,1.305767e+05,1.375244e+05,33568.736899,17255.142646,0.464946,0.268817,0.184516,0.015054,0.000430
std,0.002229,0.000909,0.004561,742.306729,0.266477,0.266153,0.243191,742.306729,0.007372,0.203451,...,1.926640e+05,2.015365e+05,7.969362e+05,77947.720054,49174.975593,1.346314,1.028633,0.527163,0.128665,0.020739
min,0.001688,0.000315,0.003550,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.153201,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.002494,0.000900,0.005455,124.000000,0.285737,0.118919,0.540621,124.000000,0.000000,0.215720,...,4.756178e+04,1.274651e+04,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.002687,0.000978,0.005960,217.000000,0.438154,0.201886,0.743220,217.000000,0.000000,0.229950,...,1.176228e+05,5.511028e+04,0.000000e+00,5211.095209,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.002922,0.001078,0.006572,440.000000,0.684235,0.386835,0.942545,440.000000,0.000000,0.248656,...,2.227261e+05,1.648135e+05,0.000000e+00,33654.810667,9705.671683,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.046621,0.019754,0.092098,20714.000000,1.000000,1.000000,1.000000,20714.000000,0.166667,1.000000,...,1.653186e+06,2.586617e+06,1.279555e+07,886875.919586,660828.326796,22.000000,26.000000,6.000000,2.000000,1.000000


In [200]:
todays_date = pd.to_datetime('today').strftime('%m%d%Y')
ct_nyc.to_csv(f'analysis_df_{todays_date}.csv', index=False)